# per ogni input controllare che l'ufficio sia presente nella lista dei codici uffici

#### Inseriamo ufficio ( tramite id )
#### Inseriamo ente ( tramite id ) numero, anno , ricerca_iqera 
#### per ora entriamo solo nella pagina non facciamo lo scraper dei risultati

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            if 'non trovato' in response2.text:
                rep = str(numero) + ' - ' + str(anno)
                return [ufficio, natura, ente, rep, '-', '-', '-','no']
            else :
                compila = 'no'
                tabella = soup2.find('div', class_='panel-body')

                if  tabella.find('span', string="Atto tassato dall'ufficio di: "): 
                    office = tabella.find('span', string="Atto tassato dall'ufficio di: ").find_next_sibling('span').get_text(strip=True)
                else :
                    office = '-'   

                if tabella.find('span', string="Tipologia atto: "):
                    atto = tabella.find('span', string="Tipologia atto: ").find_next_sibling('span').get_text(strip=True)
                else :
                    atto = '-'  

                if tabella.find('span', string="Emesso da: "):
                    emesso_da = tabella.find('span', string="Emesso da: ").find_next_sibling('span').get_text(strip=True)
                else :
                    emesso_da = '-'   

                if tabella.find('span', string="n. Repertorio - anno: "):
                    repertorio = tabella.find('span', string="n. Repertorio - anno: ").find_next_sibling('span').get_text(strip=True)
                else :
                    repertorio = '-'  

                if tabella.find('span', string="Attore: "):
                    attore = tabella.find('span', string="Attore: ").find_next_sibling('span').get_text(strip=True)
                else:
                    attore='-'

                if tabella.find('span', string="Convenuto: "):
                    convenuto = tabella.find('span', string="Convenuto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    convenuto ='-'

                if tabella.find('span', string="Stato dell'atto: "):
                    stato = tabella.find('span', string="Stato dell'atto: ").find_next_sibling('span').get_text(strip=True)
                else:
                    stato= '-'

                if 'compilaf24.php' in response2.text:  # Check if the link is present in the whole body
                    compila = 'si'



                return [office,atto,emesso_da,repertorio,attore,convenuto,stato,compila]
        else :
            rep = str(numero) + ' - ' + str(anno)
            return [ufficio, natura, ente, rep, '-', '-', '-','no']

    else:
       return 'ufficio non trovato'   

#### cambia la natura dell'ente in base al tipo diu ricerca ( se monitoria o ppt)

In [3]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

def tipo_scraper(contratto,debitore, tipo, ufficio, lista_enti, anno, numero,row_number):
    results = []
    try:
        if tipo == 'monitoria':
            natura_list = list_natura_monitoria
        elif tipo == 'ppt':
            natura_list = list_natura_ppt
        else:
            raise ValueError("Invalid tipo value")

        for natura in natura_list:
            for ente in lista_enti:
                result = scraper(ufficio, ente, anno, numero, natura)
                if isinstance(result, list):
                    if len(result) == 8:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'atto': result[1],
                            'emesso da': result[2],
                            'repertorio': result[3],
                            'attore': result[4],
                            'convenuto': result[5],
                            'stato': result[6],
                            "compila":result[7]
                        })
                    else:
                        results.append({
                            "contratto":contratto,
                            "debitore":debitore,
                            'ufficio': result[0],
                            'natura': result[1],
                            'ente': result[2],
                            'repertorio': result[3],
                            'attore': '-',
                            'convenuto': '-',
                            'stato': '-',
                            "compila":'no'
                        })
                else:
                    print(result)
    except Exception as e:
        print(f'Riga con errore: {row_number}')
        print(f"Error occurred: {e}")
    return results


#### sistemiamo l'input prima di mandare lo scaper 

In [4]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json
from datetime import date
import numpy as np
today = date.today()


In [5]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [6]:
input = pd.read_excel('Input/Raw/perimetro_03_06.xlsx')

In [7]:
n_columns = input.filter(regex='^N')
contains_dash = n_columns.applymap(lambda x: '-' in str(x) and '/' not in str(x))
n_columns = n_columns.applymap(lambda x: x.replace('-', '/') if '-' in str(x) else x)
input[n_columns.columns] = n_columns

In [8]:
input = input[input['N. RG'].str.contains('/', na=True) | input['N. RG'].isna()]
input = input[input['N. Decreto'].str.contains('/', na=True) | input['N. Decreto'].isna()]
input = input[input['N.R.G.E PPT'].str.contains('/', na=True) | input['N.R.G.E PPT'].isna()]
input = input[input['N° Repertorio'].str.contains('/', na=True) | input['N° Repertorio'].isna()]
input = input[input['N° Rep PPT'].str.contains('/', na=True) | input['N° Rep PPT'].isna()]
input = input[input['N. Cronologico'].str.contains('/', na=True) | input['N. Cronologico'].isna()]

In [9]:
input

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,N. RG,N. Decreto,N° Repertorio,N. Cronologico,N.R.G.E PPT,N° Rep PPT,RICERCA IQERA,ANNO DECRETO/PPT,Cliente,Esito Ricerca
0,EN926000168142_18012016,DI STEFANO ADELAIDE,Tribunale,SIRACUSA,NaN,NaN,NaN,NaN,1139/2019,4449/2019,PPT,2019,ENI,NaN
1,EN824247076_25012016,D'ANGELI SALVATORE,Tribunale,SIRACUSA,NaN,NaN,NaN,NaN,530/2019,838/2019,PPT,2019,ENI,NaN
2,CRS5602898,ALFANO EMILIO,Tribunale,SANTA MARIA CAPUA VETERE,Rg. 4167/2021,Nr. 1681/2021,1005/2022,1817/2022,NaN,NaN,MONITORIA,2021,MBCREDIT,NaN
3,CRS5606632,CIOTOLA PASQUALE,Tribunale,NAPOLI,Rg. 10557/2021,Nr. 4139/2021,10451/2021,11639/2021,NaN,NaN,MONITORIA,2021,MBCREDIT,NaN
4,AKCO300324863.001,ALOI ANTONINO,Tribunale,SIRACUSA,Rg. 5030/17,Nr. 2360/17,1115/2019,NaN,NaN,NaN,MONITORIA,2017,AK NORDIC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2345,TBCA0082619771.075,CUTRONEO SILVANA,Tribunale,BARCELLONA POZZO DI GOTTO,NaN,NaN,NaN,NaN,550/2019,NaN,PPT,2019,TIBERIUS,NaN
2346,TBCO0012432607.074,SATIRO SALVATORE LUCA,Tribunale,GELA,NaN,NaN,NaN,NaN,102/2020,NaN,PPT,2020,TIBERIUS,NaN
2347,XEN919002886274,IOZZO ANTONIETTA,Tribunale,Rovigo,NaN,NaN,NaN,NaN,212/2021,NaN,PPT,2021,ENI,NaN
2348,TM388268/2018_24122018,CICIRIELLO VINCENZO,Tribunale,GROSSETO,NaN,NaN,NaN,NaN,137/2021,NaN,PPT,2021,TIM,NaN


In [10]:


input = clean(input)
#input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
#input = input.drop(columns=['RICERCA IQERA'])

In [11]:


mapping = {'albenga':'savona',
    'chiavenna':'sondrio',
    'cossato':'biella',
    'macomer':'nuoro',
    'isili':'nuoro',
    'chiaromonte': 'lagonegro',
    'orzinuovi' : 'brescia',
    'ovada': 'acquiterme',
    'pavullo nel frignano':'modena',
    'ponte san pietro':'bergamo',
    'romano di lombardia':'bergamo',
    'san benedetto del tronto':'ascoli piceno',
    'teano':'caserta',
    'venezia 2 - mestre' : 'venezia',
    'venosa':'potenza',
    'vimercate':'monza',
    'acerra' : 'casoria',
    'torre annunziata':'napoli',
    'napoli nord in aversa':'napoli',
    'napoli nord':'napoli',
    'san giorgio ionico':'taranto',
    "rometta" :'barcellona pozzo di gotto',
    'taurianova':'palmi',
    'putignano':'gioia del colle',
    'dolo':'venezia',
    'ugento':'casarano',
    'grumello del monte':'bergamo',
    'todi':'perugia',
    'gragnano':'castellammare di stabia',
    'tribunale  di vicenza':'vicenza',
    'marigliano':'nola',
    "reggio nell'emilia": 'reggio emilia',
    'montoro':'avellino',
    'nocera inferiore':'pagani',
    "chieti - sezione distaccata di ortona":'chieti',
    'la maddalena':'olbia',
    'san giovanni in fiore':'cosenza',
    'poggio mirteto':'rieti',
    'volterra':'pontedera',
    'tribunale della spezia':'la spezia',
    'arcidosso':'grosseto',
    'martina franca':'taranto',
    'naso':'messina',  # forse
    'scalea':'paola',
    'cariati':'rossano',
    'montecorvino rovella':'salerno',
    "sant'anastasia":'napoli',
    'forlì':"forli'"
    }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input


In [12]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code_office': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [13]:

input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

In [14]:
input[input['code_office'].isna()]['Foro'].drop_duplicates()

Series([], Name: Foro, dtype: object)

#### sopra abbiamo aggiunto il codice dell'ufficio

#### ora dobbiamo aggiungere i codici degli enti

In [15]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [16]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)

In [17]:
# Convert the 'code_ente' lists to strings
input['code_ente'] = input['code_ente'].apply(lambda x: ','.join(x))


potential_value_vars = [
    'nr.decreto', 'year.decreto', 'nr.rg', 'year.rg',
    'nr.repertorio', 'year.repertorio', 'nr.crono', 'year.crono',
    'nr.rgeppt', 'year.rgeppt', 'nr.repppt', 'year.repppt'
]
value_vars = [col for col in potential_value_vars if col in input.columns]
# Melt the DataFrame to 'pivot longer'
df_long = pd.melt(input, 
                  id_vars=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente'], 
                  value_vars=value_vars, 
                  var_name='type', 
                  value_name='value')

# Split the 'type' column into 'attribute' and 'category'
df_long[['attribute', 'category']] = df_long['type'].str.split('.', expand=True)

# Pivot the DataFrame to get the desired format
input = df_long.pivot_table(index=['NR_Contratto', 'Debitore','Tribunale/Gdp','Foro','Cliente','tipo_ricerca','code_office','office','code_ente','category'], 
                               columns='attribute', 
                               values='value', 
                               aggfunc='first').reset_index()

# Flatten the columns
input.columns.name = None
input = input.reset_index(drop=True)

# Convert the 'code_ente' column back into a list
input['code_ente'] = input['code_ente'].apply(lambda x: x.split(',') if isinstance(x, str) else x)


In [33]:
input.to_excel('Input/Cleaned/2024_06_03_input_v1.xlsx',index=False)

In [18]:
input

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,tipo_ricerca,code_office,office,code_ente,category,nr,year
0,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",crono,4693,2020
1,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",decreto,1236,2020
2,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TM4,brescia ut dpbs,"[003, 005, 006, 009]",rg,1236,2020
3,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",crono,4693,2020
4,AA1200001768_06112018,SGOBBA IVAN,giudice,brescia,A2A,monitoria,TMJ,brescia 1 ut dpbs,"[003, 005, 006, 009]",decreto,1236,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
5817,XUB385803,STOP CAFFE' SNC DI GENNARELLI GENNARO E GUARAG...,tribunale,civitavecchia,Sirio,monitoria,TJF,civitavecchia - ante 01.02.2015 ut dprm1,"[001, 004, 005, 006, 007, 011]",rg,193,2023
5818,XUB386298,PESCE MAURO,tribunale,pescara,Sirio,monitoria,TAU,pescara ut dppe,"[001, 005, 006]",decreto,4962,2021
5819,XUB386298,PESCE MAURO,tribunale,pescara,Sirio,monitoria,TAU,pescara ut dppe,"[001, 005, 006]",rg,4962,2021
5820,XUB388609,MERLINO TETTI S.R.L.,giudice,crema,Sirio,monitoria,TMZ,crema ut dpcr,[002],decreto,566,2023


In [19]:
new_df = input[2000:2500]
results =( new_df 
    .apply(lambda row: tipo_scraper(contratto=row['NR_Contratto'],
                                    debitore=row['Debitore'],
                                    tipo= row['tipo_ricerca'],
                                    ufficio= row['code_office'],
                                    lista_enti=row['code_ente'], anno=row['year'],numero= row['nr'],
                                    row_number=row.name
                                    ),
                                    axis=1)
)

Risultati in un df

In [32]:
filtered_data = [item for item in results if item != []]
flattened_results = [item for sublist in filtered_data for item in (sublist if isinstance(sublist, list) else [sublist])]
df = pd.DataFrame(flattened_results)
df = df.drop_duplicates()
df

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
0,EL34020/SEN/2018_19022020,LDP GROUP INDUSTRY,COMO UT DPCO,DECRETO INGIUNTIVO n. 162 del 2021,GIUDICE DI PACE DI COMO,2356/20 - 2021,ARTSANA - SOCIETA' PER AZIONI,-,ATTO REGISTRATO,no
1,EL34020/SEN/2018_19022020,LDP GROUP INDUSTRY,TMX,DI,004,162 - 2021,-,-,-,no
2,EL34020/SEN/2018_19022020,LDP GROUP INDUSTRY,TMX,AV,002,162 - 2021,-,-,-,no
3,EL34020/SEN/2018_19022020,LDP GROUP INDUSTRY,TMX,AV,004,162 - 2021,-,-,-,no
8,EL34582/SEN/2018_13112018,DE SANTIS SAVERIO,TJP,DI,011,51096 - 2019,-,-,-,no
...,...,...,...,...,...,...,...,...,...,...
5633,IRGR11013690_11032020,SIDIBE SAMBA TENIN,TNH,AV,007,156 - 2021,-,-,-,no
5640,IRGR15261392_07022020,RICAGNO PAOLO,ACQUI TERME UT DPAL,DECRETO INGIUNTIVO n. 485 del 2022,GIUDICE DI PACE ACQUI TERME,68/2024 - 2022,IRETI S.P.A.,RICAGNO,IN ATTESA DI PAGAMENTO,si
5641,IRGR15261392_07022020,RICAGNO PAOLO,TSD,DI,004,485 - 2022,-,-,-,no
5642,IRGR15261392_07022020,RICAGNO PAOLO,TSD,AV,003,485 - 2022,-,-,-,no


Righe con risultati

In [33]:
df[(df['attore'] != '-') | (df['convenuto'] != '-') ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
0,EL34020/SEN/2018_19022020,LDP GROUP INDUSTRY,COMO UT DPCO,DECRETO INGIUNTIVO n. 162 del 2021,GIUDICE DI PACE DI COMO,2356/20 - 2021,ARTSANA - SOCIETA' PER AZIONI,-,ATTO REGISTRATO,no
24,EL3475/SEN/2016_19022020,C C DI CIPRIANI ARCANGELO E C SAS,PAVIA UT DPPV,DECRETO INGIUNTIVO n. 1711 del 2020,TRIBUNALE,1711 - 2020,PRELIOS CREDIT SERVICING S.P.A,DELFINA ZICHELLA,ATTO REGISTRATO,no
28,EL36510/ENE/2018_19022020,ECHCHAHED ABDERRAZAK,BERGAMO 1 UT DPBG,DECRETO INGIUNTIVO n. 205 del 2021,GIUDICE DI PACE DI BERGAMO,205 - 2021,ALARM SYSTEM S.R.L.,PONTIGGIA,ATTO REGISTRATO,no
30,EL36510/ENE/2018_19022020,ECHCHAHED ABDERRAZAK,BERGAMO 1 UT DPBG,DECRETO INGIUNTIVO n. 205 del 2021,GIUDICE DI PACE GRUMELLO,205 - 2021,CONDOMINIO PRIMAVERA,VALLI,ATTO REGISTRATO,no
104,EL5968/SEN/2015_19022020,ROSSI BRUNO,MANTOVA UT DPMN,DECRETO INGIUNTIVO n. 895 del 2020,GIUDICE DI PACE-MANTOVA,502 - 2022,SERVIZIO ELETTRICO NAZIONALE S,ROSSI,ATTO IN FASE DI TASSAZIONE,no
...,...,...,...,...,...,...,...,...,...,...
5573,IRGE13820481_31102019,SIBY MOUSSA,TORINO 1 UT DPTO1,ESECUZIONE MOBILIARE n. 1863 del 2021,TRIBUNALE TORINO,3340 - 2021,IREN MERCATO,SIBY,ATTO REGISTRATO,no
5588,IRGE4120319_20042018,BILCU ROBERT,TORINO 1 UT DPTO1,ORDINANZA n. 3386 del 2021,TRIBUNALE TORINO,3297 - 2022,INTESA SANPAOLO S.P.A.,BE DESIGN INTERNATIONAL SRL,ATTO REGISTRATO,no
5600,IRGE5099754_20042018,PERRONE DANIELE,ALESSANDRIA UT DPAL,ESECUZIONE MOBILIARE n. 703 del 2021,TRIBUNALE,703 - 2021,GEO-EDIL S.R.L.,BERCHIO,ATTO REGISTRATO,no
5624,IRGE6360544_19122017,ROTELLA GIUSEPPE,IVREA UT DPTO2,ESECUZIONE MOBILIARE n. 965 del 2020,TRIBUNALE DI IVREA,956/2020 - 2020,IREN MERCATO,ROTELLA,EMESSO ARTICOLO DI RUOLO,no


Guardare i nomi in format 'name +1'

In [34]:
df[(df['attore'].str.contains('\+')) | (df['convenuto'].str.contains('\+')) ]

,contratto,debitore,ufficio,atto,emesso da,repertorio,attore,convenuto,stato,compila
248,EL8188/ENE/2016_19022020,DIGLIO VERONICA ANNA,MILANO 1 UT DPMI1,DECRETO INGIUNTIVO n. 14924 del 2020,TRIB. CIVILE ORDINARIO,10682 - 2020,VACCARO MILVA,MERCADANTE CRISTINA+1,ATTO REGISTRATO,no


Toglie il +1 a chi lo ha

In [35]:
mask_attore = df['attore'].str.contains('\+')
mask_convenuto = df['convenuto'].str.contains('\+')

df.loc[mask_attore, 'attore'] = df.loc[mask_attore, 'attore'].str.split('\+').str[0].str.strip()
df.loc[mask_convenuto, 'convenuto'] = df.loc[mask_convenuto, 'convenuto'].str.split('\+').str[0].str.strip()


In [36]:
# df = pd.DataFrame(results.explode().tolist())
df.to_excel(f'Output/{str(today)}_output_totale_2500_v1.xlsx',index=False)


In [37]:
mask = df.apply(lambda row: (row['attore'] in row['debitore']) or (row['convenuto'] in row['debitore']), axis=1)

df[mask].to_excel(f'Output/{str(today)}_esiti_positivi_2500_v1.xlsx',index=False)